<h1>Data Preparation</h1>

<h2>Objective</h2>
The objective of this notebook is to prepare the online retail dataset for clustering analysis. The steps include loading, exploring, cleaning, engineering features, transforming the data, and finally saving the cleaned dataset. This data preparation pipeline will help identify customer segments using unsupervised learning techniques like K-means clustering.

In [10]:
import numpy as np
import pandas as pd

<h3>1. Loading Data</h3>

In [11]:
excel_path = r"C:\Users\Admin\project\Online Retail.xlsx"
dataset = pd.read_excel(excel_path)
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


<h3>2. Exploring Data</h3>

In [12]:
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None


In [14]:
# Statistical summary (for numerical columns)
print(dataset.describe())

            Quantity                    InvoiceDate      UnitPrice  \
count  541909.000000                         541909  541909.000000   
mean        9.552250  2011-07-04 13:34:57.156386048       4.611114   
min    -80995.000000            2010-12-01 08:26:00  -11062.060000   
25%         1.000000            2011-03-28 11:34:00       1.250000   
50%         3.000000            2011-07-19 17:17:00       2.080000   
75%        10.000000            2011-10-19 11:27:00       4.130000   
max     80995.000000            2011-12-09 12:50:00   38970.000000   
std       218.081158                            NaN      96.759853   

          CustomerID  
count  406829.000000  
mean    15287.690570  
min     12346.000000  
25%     13953.000000  
50%     15152.000000  
75%     16791.000000  
max     18287.000000  
std      1713.600303  


In [15]:
print(dataset.isna().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


<h3>3. Cleaning Data</h3>

In [16]:
filtered_data = dataset.dropna(subset=['Description']).copy()
print(filtered_data.isna().sum())

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133626
Country             0
dtype: int64


In [19]:
# Fill missing CustomerID with 'Unknown'
filtered_data['CustomerID'] = filtered_data['CustomerID'].fillna('Unknown')

In [20]:
# Drop duplicate records from the dataset
filtered_data = filtered_data.drop_duplicates()

print(f"Rows left after eliminating duplicates: {filtered_data.shape[0]}")


Rows left after eliminating duplicates: 535187


In [21]:
# Convert InvoiceDate to datetime format
filtered_data['InvoiceDate'] = pd.to_datetime(filtered_data['InvoiceDate'])

# Cast CustomerID to string type
filtered_data['CustomerID'] = filtered_data['CustomerID'].astype(str)

print(filtered_data.dtypes)

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object


<h3>4. Feature Engineering</h3>

In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Scale numerical features 'Quantity' and 'UnitPrice'
filtered_data[['Quantity_scaled', 'UnitPrice_scaled']] = scaler.fit_transform(filtered_data[['Quantity', 'UnitPrice']])

print(filtered_data[['Quantity', 'Quantity_scaled', 'UnitPrice', 'UnitPrice_scaled']].head())


   Quantity  Quantity_scaled  UnitPrice  UnitPrice_scaled
0         6        -0.016761       2.55         -0.021520
1         6        -0.016761       3.39         -0.012892
2         8        -0.007631       2.75         -0.019465
3         6        -0.016761       3.39         -0.012892
4         6        -0.016761       3.39         -0.012892


In [23]:
# Apply one-hot encoding to 'Country', avoiding multicollinearity
encoded_data = pd.get_dummies(filtered_data, columns=['Country'], drop_first=True)

print(f"Data shape after encoding: {encoded_data.shape}")

Data shape after encoding: (535187, 46)


In [24]:
# Generate 'TotalAmount' by multiplying quantity with unit price
encoded_data['TotalAmount'] = encoded_data['Quantity'] * encoded_data['UnitPrice']

print(encoded_data[['Quantity', 'UnitPrice', 'TotalAmount']].head())

   Quantity  UnitPrice  TotalAmount
0         6       2.55        15.30
1         6       3.39        20.34
2         8       2.75        22.00
3         6       3.39        20.34
4         6       3.39        20.34


In [25]:
# Create YearMonth period from InvoiceDate for temporal analysis
encoded_data['YearMonth'] = encoded_data['InvoiceDate'].dt.to_period('M')

print(encoded_data[['InvoiceDate', 'YearMonth']].head())


          InvoiceDate YearMonth
0 2010-12-01 08:26:00   2010-12
1 2010-12-01 08:26:00   2010-12
2 2010-12-01 08:26:00   2010-12
3 2010-12-01 08:26:00   2010-12
4 2010-12-01 08:26:00   2010-12


In [26]:
# Extract hour component from InvoiceDate
encoded_data['InvoiceHour'] = encoded_data['InvoiceDate'].dt.hour

print(encoded_data[['InvoiceDate', 'InvoiceHour']].head())


          InvoiceDate  InvoiceHour
0 2010-12-01 08:26:00            8
1 2010-12-01 08:26:00            8
2 2010-12-01 08:26:00            8
3 2010-12-01 08:26:00            8
4 2010-12-01 08:26:00            8


In [27]:
# Calculate correlation matrix for numerical features
correlation_matrix = encoded_data.corr(numeric_only=True)

# Display correlations with TotalAmount
print(correlation_matrix['TotalAmount'].sort_values(ascending=False))


TotalAmount                     1.000000
Quantity                        0.888196
Quantity_scaled                 0.888196
Country_Netherlands             0.017831
Country_Japan                   0.005467
Country_Sweden                  0.004713
Country_EIRE                    0.004565
Country_Denmark                 0.002128
Country_Germany                 0.001829
Country_Norway                  0.001681
Country_France                  0.001647
Country_Switzerland             0.001619
Country_Finland                 0.001320
Country_Singapore               0.001175
Country_Hong Kong               0.001010
Country_Channel Islands         0.000824
Country_Spain                   0.000630
Country_Lithuania               0.000621
Country_Greece                  0.000611
Country_Israel                  0.000535
Country_Austria                 0.000514
Country_Lebanon                 0.000468
Country_Brazil                  0.000356
Country_Italy                   0.000291
Country_United A

In [28]:
# Save the cleaned dataset to a CSV file
filtered_data.to_csv("cleaned_retail_data.csv", index=False)


<h3>Conclusion</h3>

This notebook focused on thorough preprocessing of the online retail dataset. Key steps covered importing the data, exploring its structure, handling missing values, creating new features, and applying scaling transformations. The refined dataset is now well-prepared for subsequent clustering tasks.

With this foundation, the upcoming analysis aims to discover distinct customer groups, enabling the business to enhance marketing efforts and deliver more personalized experiences.